<a href="https://colab.research.google.com/github/AnamikaMangore/anamika-ka-project/blob/main/POV_Alexa_Rule_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stanza sacrebleu evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are install

In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bidict                           0.23.1

In [ ]:
## Check for cuda
!nvidia-smi

Thu Jul 11 16:40:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
## Imports
import random
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import spacy
import stanza
import evaluate
from spacy import displacy
from nltk.tokenize import word_tokenize

# set options
pd.set_option('max_colwidth', None) # show full text
random.seed(42)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


path="/content/drive/MyDrive/alexa-point-of-view-dataset-main"

In [ ]:
## Data
train = pd.read_csv(f"/content/drive/MyDrive/alexa-point-of-view-dataset-main/data/train.tsv", sep="\t", dtype={"input": str, "output": str})
test = pd.read_csv(f"/content/drive/MyDrive/alexa-point-of-view-dataset-main/data/test.tsv", sep="\t", dtype={"input": str, "output": str})
dev = pd.read_csv(f"/content/drive/MyDrive/alexa-point-of-view-dataset-main/data/dev.tsv", sep="\t", dtype={"input": str, "output": str})
total = pd.read_csv(f"/content/drive/MyDrive/alexa-point-of-view-dataset-main/data/total.tsv", sep="\t", dtype={"input": str, "output": str})
print("Train", train.shape)
print("Test", test.shape)
print("Dev", dev.shape)
print("Total", total.shape)

## Preprocess
train.input = train.input.str.replace("@CN@","Bob")
train.output = train.output.str.replace("@CN@","Bob")
train.output = train.output.str.replace("@SCN@", "John")

train.reset_index(drop=False, inplace=True)
train.head(10)


In [ ]:
# Accessing the new index column
first_index_value = train.at[5, 'index']

# print(first_index_value)
print(train.input[first_index_value])

In [ ]:
train= train[0:7000]

### Classifying questions using rules

There are 4 message types invovled here:

- AskWH messages: includes wh-words such as who, what, when, where
- AskYN messages: includes phrases such as ask if, ask whether or questions starting with are, is, can
- Req messages: includes phrases like tell to, ask to, remind to, etc.
- Statement messages: includes tell that, message that, remind that

### Lets check if by checking of these above words we can cover all the question types in the data or not

In [ ]:
ask_wh = ["who", "what", "when", "where"]
ask_yn = ["ask if", "ask whether"]
ask_yn_starts_with = ("is", "are", "can", "could", "will")
ask_req = ["tell.*to", "ask.*to", "remind.*to", "ask.*for"]
statements = ["tell.*that", "message.*that", "remind.*that", "know.*that"]

### Ask WH Questions

In [ ]:
## Identify WH questions
##print('|'.join(ask_wh))
train_ask_wh_questions = train[train.input.str.contains('|'.join(ask_wh), regex=True)]
print("Ask WH Questions", train_ask_wh_questions.shape[0])
train_ask_wh_questions

### Ask YN Questions

In [ ]:
## Identify YN questions
train_ask_yn_questions_1 = train[train.input.str.contains('|'.join(ask_yn), regex=True)]
train_ask_yn_questions_2 = train[train.input.str.startswith(ask_yn_starts_with)]
train_ask_yn_questions = pd.concat([train_ask_yn_questions_1, train_ask_yn_questions_2])
print("Ask YN Questions", train_ask_yn_questions.shape[0])
train_ask_yn_questions

### Ask Req Questions

In [ ]:
## Identify Req questions
train_ask_req_questions = train[train.input.str.contains('|'.join(ask_req), regex=True)]
print("Ask Req Questions", train_ask_req_questions.shape[0])
train_ask_req_questions

### Statements

In [ ]:
## Identify Statements
train_ask_stmt_questions = train[train.input.str.contains('|'.join(statements), regex=True)]
print("Ask Statement Questions", train_ask_stmt_questions.shape[0])
train_ask_stmt_questions

### Check if we have any common question in these categories

In [ ]:
train_ask_wh_questions.index.intersection(train_ask_yn_questions.index).shape

In [ ]:
train_ask_req_questions.index.intersection(train_ask_wh_questions.index).shape

In [ ]:
train_ask_req_questions.index.intersection(train_ask_yn_questions.index).shape

In [ ]:
train_ask_stmt_questions.index.intersection(train_ask_yn_questions.index).shape

In [ ]:
train_ask_stmt_questions.index.intersection(train_ask_req_questions.index).shape

In [ ]:
train_ask_stmt_questions.index.intersection(train_ask_wh_questions.index).shape

As we can observe there are many common question in Ask WH and Req messages, the paper uses TFIDF and a custom list of stop words to reduce these number of common questions, but for now I will ignore them and perform processing for each one of them individually.

In [ ]:
train["ask_wh"] = train.input.str.contains('|'.join(ask_wh), regex=True)
train["ask_yn"] = train.input.str.contains('|'.join(ask_yn), regex=True) | train.input.str.startswith(ask_yn_starts_with)
train["ask_req"] = train.input.str.contains('|'.join(ask_req), regex=True)
train["stmt"] = train.input.str.contains('|'.join(statements), regex=True)
train.head()

### Total how many of these question from the total were we able to classify?

In [ ]:
len(train_ask_wh_questions) + len(train_ask_yn_questions) + len(train_ask_req_questions) + len(train_ask_stmt_questions), len(train)

### POS Tagging

The POS tagging will help identify direct questions from indirect ones, the indriect question come with the label SQ tagging. The sentence level tags (S, S', SQ) are crucial in determining whether the word order between the subject and the auxillary is reversed. The word level tags (VB, VBP, VBZ0 indicate which verb needs to be changed as part of POV conversion.

In [ ]:
## NLTK POS Tagging
random_sentence = random.choice(train["input"].values)

words = nltk.word_tokenize(random_sentence)

tagging = nltk.pos_tag(words)
print(tagging)

In [ ]:
## Standford nlp
stanford_nlp = stanza.Pipeline('en')
stanford_doc = stanford_nlp(random_sentence)

In [ ]:
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in stanford_doc.sentences for word in sent.words], sep='\n')

In [ ]:
## Spacy
nlp = spacy.load("en_core_web_sm")
# doc = nlp(random_sentence)
doc = nlp("ask Bob, when are you coming for dinner")
for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_)
displacy.render(doc, style = "dep", jupyter=True)

### Constituent Parsing

| POS Tag | Info |

| S | Simple Declarative Clause |

| SBAR (S') | Clause introduced by subordinating conjunction |

| SQ | Inverted yes/no question |

| VB | verb, base form |

| VBP | verb, non-3rd person singular present |

| VBZ | verb, third person singular present |

| VP | verb phrase |

In [ ]:
## Constituency parsing
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')
# random_sentence = random.choice(train["input"].values)
random_sentence = "ask Bob, when are you coming for dinner"
doc = nlp(random_sentence)
for sentence in doc.sentences:
    print(sentence.constituency)

In [ ]:
## Constituency tree - Direct example
tree = doc.sentences[0].constituency
tree

In [ ]:
## Constituency tree - Indirect example
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')
doc_2 = nlp("ask Bob, when he is coming for dinner")
tree_2 = doc_2.sentences[0].constituency
print("Tree:")
tree_2

In [ ]:
## Check if it's a direct or indirect question
"SQ" in str(tree_2), "SQ" in str(tree)

In [ ]:
## Run over all examples to classify direct and indirect questions
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency', verbose=False, use_gpu=True)
def check_direct(text):
  doc = nlp(text)
  tree = doc.sentences[0].constituency
  return "SQ" in str(tree)

train["is_direct"] = train.input.apply(lambda x: check_direct(str(x)))
train.head()

In [ ]:
input_values = train.input.values[:7000]
train["is_direct"] = [check_direct(str(x)) for x in input_values]
train.head()

In [ ]:
train.shape

In [ ]:
# import stanza
# nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency', verbose=False, use_gpu=True)
# out_docs = nlp.bulk_process(train.input.values) # Call the neural pipeline on this list of documents
# print(out_docs[1]) # The output is also a list of stanza.Document objects, each output corresponding to an input Document object

In [ ]:
train.is_direct.value_counts()

In [ ]:
train.to_csv(f"{path}/train_processed.csv", index=False)

In [ ]:
train_processed = pd.read_csv(f"{path}/train_processed.csv")
print(train_processed.shape)
train_processed.head()

### Transformations

1. `Changing word order`:

This step only applies to direct questions in AskYN and AskWH messages. During this process, multiple types of grammatical changes may apply, including do-deletion, and subject-auxiliary reversal (are you → you are).


In [ ]:
## Reverse the subject-auxillary order
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Example text message
random_direct_yn_row = train_processed.iloc[927]

print("Original Message:", random_direct_yn_row.input)
print("Expected response:", random_direct_yn_row.output)

def change_word_order(sentence):
  # Tokenize the text
  doc = nlp(sentence)

  # Process each sentence
  for sentence in doc.sents:
      subject = None
      verb = None
      verb_form = None

      # Identify the subject and verb
      for token in sentence:
          if ("subj" in token.dep_) or ("nsubj" in token.dep_):
              subject = token
          if ("aux" in token.dep_) and (token.text.lower() in ["is", "are"]):
              verb = token
              verb_form = token.text.lower()
          if not verb and ("ccomp" in token.dep_):
              verb = token
              verb_form = token.text.lower()

      # Check for subject-verb disagreement and reorder if needed
      if subject and verb and (verb_form in ["is", "are"]):
          # Swap the subject and verb
          new_sentence = f"hey Bob, {subject} {verb} {sentence[max(subject.i, verb.i) + 1:]}"
          transformed_sentence = new_sentence
      else:
          # No correction needed
          transformed_sentence = sentence.text
  return transformed_sentence

transformed_sentence = change_word_order(random_direct_yn_row.input)
print("Transformed sentence:", transformed_sentence)

In [ ]:
direct_yn_ids = [24, 25, 5043, 3258, 1801, 2452]
for sent in train_processed.loc[direct_yn_ids].itertuples():
  print("Org:",sent.input)
  print("Exp:", sent.output)
  print("Transformed:", change_word_order(sent.input))
  print()

2. `Swapping pronouns/contact names`:

We use rules to convert a first person (I) to a third person (he/she) and a third person (he/she) to a second person (you). In cases where the contact name resides inside the message content, the rules would find it and switch it with a second person pronoun.

In [ ]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Example text message
random_wh_row = train_processed.iloc[2497]

print("Original Message:", random_wh_row.input)
print("Expected response:", random_wh_row.output)

def swap_pronouns(sentence):
  # Tokenize the text
  doc = nlp(sentence)

  # Define a set of pronouns to consider
  first_person_pronouns = {"I": "he", "me": "him", "my": "his"}
  third_person_pronouns = {"he": "you", "she": "you", "him": "you", "his": "your", "hers": "you"}

  # Apply rules to modify pronouns
  modified_text = []
  for token in doc:
      if token.text in first_person_pronouns:
          modified_text.append(first_person_pronouns[token.text])
      elif token.text in third_person_pronouns:
          modified_text.append(third_person_pronouns[token.text])
      else:
          modified_text.append(token.text)

  # Generate the modified text
  modified_text_message = " ".join(modified_text)
  return modified_text_message

transformed_sentence = swap_pronouns(change_word_order(random_wh_row.input))
print("Transformed sentence:", transformed_sentence)

3. `Fixing verb agreement`:

This step is to make sure the main verb/auxiliary agrees with the converted subject pronoun in person and number. In sentences with present tense, if we switch the subject she to you, we must change the main verb to its base form (is → are, wants → want, VBZ → VBP) as well.

In [ ]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Example text message
text_message = "can you ask if she wants icecream ?"

def fix_verb_agreement(sentence):
  # Tokenize the text
  doc = nlp(sentence)


  verb_agreement_rules = {
      "is": "are",
      "wants": "want",
      "has": "have"
  }

  if "you" in sentence:
      for word, change in verb_agreement_rules.items():
        sentence = sentence.replace(word.lower(), change)

print("Org:", text_message)
fix_verb_agreement(text_message)

4. `Adding prepending rules to reconstructed message content`:

We add the source contact name and appropriate reporting verbs to the beginning of each output, among other things. Each type of message has a different set of prepend rules and the VA can randomly choose prepends in the same set to sound more spontaneous. For example, an AskYN message with a direct question would need a prepend rule like@SCN@ asks if or @SCN@ is wondering whether. Similarly, a Req messages might use @SCN@ asks you to or @SCN@ would like to remind you to as prepends.

In [ ]:
import random

# Example text message
random_wh_row = train_processed.iloc[3269]

print("Original Message:", random_wh_row.input)
print("Expected response:", random_wh_row.output)

def extract_message(text, prepend_suggestion):
  message = random.choice(["Hey", "Hi"]) + " Bob, John " + prepend_suggestion
  # text = str(text.replace("@CN@", "John")).replace("to", "").replace("that","")
  doc = nlp(text)
  end_index = [e.end_char for e in doc.ents if e.label_ == "PERSON"]
  if len(end_index) > 0:
    message += text[end_index[0]:].replace(",","")
  return message

def reconstruct_message(sentence, message_types):
  # Define prepend rules for different message types
  prepend_suggestions = [""]
  if message_types.ask_yn and message_types.is_direct:
    prepend_suggestions = ["asks if", "is wondering whether", "wants to know if", "would like to know whether"]
  elif message_types.ask_req:
    prepend_suggestions = ["asks you to", "would like to remind you to", "requests that"]
  elif message_types.stmt:
    prepend_suggestions = ["shares that", "provides information that", "tells you that"]

  # Randomly select a prepend from the rules for the given message type
  selected_prepend = random.choice(prepend_suggestions)

  # Extract the message
  message = extract_message(sentence, selected_prepend)

  return message

modified_sentence = reconstruct_message(swap_pronouns(change_word_order(random_wh_row.input)), random_wh_row)
print("Transformed:", modified_sentence)

In [ ]:
## Apply all the preprocessing sequentially

# Step 1:
train_processed["transformed"] = train_processed.apply(lambda row: change_word_order(row.input) if row['ask_yn'] and row['is_direct'] else row.input, axis=1)
train_processed["transformed"] = train_processed.input.apply(swap_pronouns)
train_processed["transformed"] = train_processed.input.apply(fix_verb_agreement)
train_processed["transformed"] = train_processed.apply(lambda row: reconstruct_message(row.input, row), axis=1)
train_processed.sample(10)

In [ ]:
# Save the transformations
train_processed.to_csv(f"{path}/train_processed.csv", index=False)

In [ ]:
pd.read_csv(f"{path}/train_processed.csv").sample(10)

### Check the results for rule-based approach

In [ ]:
metric = evaluate.load("sacrebleu")

metric.compute(predictions=train_processed.transformed.values, references=train_processed.output.values)

In [ ]:
import random
random_row = train_processed.loc[random.randint(0,len(train_processed))]
predictions = [random_row.transformed]
references = [[random_row.output]]
print("Input:", random_row.input)
print("Output:", random_row.output)
print("Rule based output:", random_row.transformed)
metric.compute(predictions=predictions, references=references)

In [ ]:
train_processed["bleu_score"] = train_processed.apply(lambda row: metric.compute(predictions=[row["transformed"]], references=[[row["output"]]])["score"], axis=1)

In [ ]:
train_processed.to_csv(f"{path}/train_processed.csv", index=False)

In [ ]:
train_processed = pd.read_csv(f"{path}/train_processed.csv")
train_processed.head()

In [ ]:
## Distribution of BLEU scores for rule-based approach
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.histplot(train_processed.bleu_score, kde=True)
plt.subplot(1,2,2)
sns.boxplot(train_processed, y="bleu_score");

In [ ]:
## Sort by the BLEU score
train_processed_sorted = train_processed.sort_values("bleu_score", ascending=False)

### Check examples that went wrong

In [ ]:
train_processed_sorted.tail(20)